In [1]:
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt

In [2]:
rbc_l=4.5
rbc_h=6.3
hb_l=13.2
hb_h=16.6
wbc_l=3.4
wbc_h=9.6
leuk_l=62
leuk_h=74
lym_l=33
lym_h=45
plt_l=150
plt_h=450
crea_l=0.7
crea_h=1.3
gluc_l=65
gluc_h=110
na_l=135
na_h=145
k_l=3.5
k_h=5.0
ca_l=1.03
ca_h=1.23
tsh_l=1.3
tsh_h=4.0
t3_l=0.2
t3_h=0.5
t4_l=0.7
t4_h=1.4

In [3]:
dic={
    'rbc':[rbc_l,rbc_h],
    'hb':[hb_l,hb_h],
    'wbc':[wbc_l,wbc_h],
    'plt':[plt_l,plt_h],
    'crea':[crea_l,crea_h],
    'gluc':[gluc_l,gluc_h],
    'na':[na_l,na_h],
    'k':[k_l,k_h],
    'ca':[ca_l,ca_h],
    'tsh':[tsh_l,tsh_h],
    't4':[t4_l,t4_h],
    't3':[t3_l,t3_h],
  
}

In [4]:
reldic={
    
    'leuk':['wbc',leuk_l,leuk_h],
    'lym':['wbc',lym_l,lym_h]
}

In [5]:
num_months=21*12


In [8]:
def random_change(vector, mean, std,length):
    mul=np.random.normal(mean, std, size=length)
    return vector * mul

def gradual_change(vector, mean, std,length):
    inc=np.random.normal(mean,std, size=length)
    inc=np.sort(inc)
    return vector + inc

In [48]:
def create_patient(id_num, sick_probs,thyr_prob):
    pat=pd.DataFrame()
    pat['month']=np.arange(0,num_months)
    pat['id']=f'pat_{id_num}'
    
    
    for lab in dic.keys():
        low=dic[lab][0]
        high=dic[lab][1]
        mean= np.random.uniform(low,high)
        std=(high-low)/4
        results=np.random.normal(mean, std, size=num_months)
        pat[lab]=results
    
    for lab in reldic:
        low=reldic[lab][1]
        high=reldic[lab][2]
        mean= np.random.uniform(low,high)
        std=(high-low)/4

        percents= np.random.normal(mean,std,num_months)
        results=percents*pat[reldic[lab][0]]/100
        pat[lab]=results
        
    
    sick=np.random.choice(np.arange(0, 3), p=sick_probs)
    sick_mult=[0.5,1,2]
    sick_start=[int(num_months*0.75),int(num_months*0.5),int(num_months*0.25)]
    viral_probs=[10/num_months, 30/num_months, 50/num_months]

    sick_p=viral_probs[sick]
    pat['sick']=np.random.choice(2,size=num_months, p=[1-sick_p,sick_p])
    
    diseases={
    'diabetes':0.3,
    'crf':0.2,
    'anemia':0.3,
    'thyroid':thyr_prob
    }
    
    for disease in diseases.keys():
        p=diseases[disease]*sick_mult[sick]
        if p>=1:
            p=1
        booli=np.random.choice(np.arange(0, 2),p=[1-p,p] )
        if booli ==0 :
            pat[disease]=0
        if booli==1:
            start=random.randint(sick_start[sick], num_months)
            pat[disease]=0
            pat.loc[pat.month >= start, disease] = 1
    
    mask = (pat.sick == 1)
    sick_len=pat[pat.sick==1].shape[0]

    pat.loc[mask,'wbc'] = random_change(pat.loc[mask,'wbc'], 2.5,0.5, sick_len)
    pat.loc[mask,'leuk'] = random_change(pat.loc[mask,'leuk'], 2.5,0.5, sick_len)
    pat.loc[mask,'plt'] = random_change(pat.loc[mask,'plt'], 1.2,0.2, sick_len)
    pat.loc[mask,'gluc'] = random_change(pat.loc[mask,'gluc'], 1.5,0.3, sick_len)

    mask = (pat.crf == 1)
    crf_len=pat[pat.crf==1].shape[0]

    pat.loc[mask,'crea'] = gradual_change(pat.loc[mask,'crea'], 0.15,0.05, crf_len)
    pat.loc[mask,'k'] = gradual_change(pat.loc[mask,'k'], 0.1,0.03, crf_len)
    pat.loc[mask,'hb'] = random_change(pat.loc[mask,'hb'], 0.9,0.04, crf_len)
    pat.loc[mask,'rbc'] = random_change(pat.loc[mask,'rbc'], 0.9,0.04, crf_len)

    mask = (pat.diabetes == 1)
    diabetes_len=pat[pat.diabetes==1].shape[0]

    pat.loc[mask,'gluc'] = gradual_change(pat.loc[mask,'gluc'], 7,0.5, diabetes_len)
    pat.loc[mask,'crea'] = gradual_change(pat.loc[mask,'crea'], 0.1,0.03, diabetes_len)

    mask = (pat.anemia == 1)
    anemia_len=pat[pat.anemia==1].shape[0]

    pat.loc[mask,'hb'] = gradual_change(pat.loc[mask,'hb'], -0.5,0.03, anemia_len)
    pat.loc[mask,'rbc'] = gradual_change(pat.loc[mask,'rbc'], -0.25,0.015, anemia_len)
    
    mask = (pat.thyroid == 1)
    thyroid_len=pat[pat.thyroid==1].shape[0]

    pat.loc[mask,'tsh'] = gradual_change(pat.loc[mask,'tsh'], 0.2,0.05, thyroid_len)
    pat.loc[mask,'t4'] = gradual_change(pat.loc[mask,'t4'], -0.1,0.05, thyroid_len)
    pat.loc[mask,'t3'] = gradual_change(pat.loc[mask,'t3'], -0.05,0.025, thyroid_len)

    pat['total']=pat.sick+pat.diabetes+pat.crf+pat.anemia+pat.thyroid
    
    mask_0 = (pat.sick == 0)
    mask_1 = ((pat.sick >0)&(pat.sick<3))
    mask_2 = (pat.sick >=3)

    drop0_len=pat[mask_0].shape[0]
    drop1_len=pat[mask_1].shape[0]
    drop2_len=pat[mask_2].shape[0]

    p_drop0=0.7
    p_drop1=0.6
    p_drop2=0.4

    for lab in ['hb','crea','tsh']:

        if lab=='hb':
            delta=0
        elif lab=='crea':
            delta=0.07
        elif lab=='tsh':
            delta=0.15

        p_drop0=p_drop0+delta
        p_drop1=p_drop1+delta
        p_drop2=p_drop2+delta


        drop0=np.random.choice(2,size=drop0_len, p=[p_drop0,1-p_drop0])
        drop1=np.random.choice(2,size=drop1_len, p=[p_drop1,1-p_drop1])
        drop2=np.random.choice(2,size=drop2_len, p=[p_drop2,1-p_drop2])

        pat.loc[mask_0,lab]=pat.loc[mask_0,lab]*drop0
        pat.loc[mask_1,lab]=pat.loc[mask_1,lab]*drop1
        pat.loc[mask_2,lab]=pat.loc[mask_2,lab]*drop2

        pat[lab].replace(0, np.nan, inplace=True)
        
    hb_cols=['rbc', 'wbc', 'plt','leuk', 'lym']
    crea_cols=['gluc', 'na', 'k','ca']
    tsh_cols=['t4','t3']
    
    for col in hb_cols:
        mask_cbc = (pat.hb.isna())
        pat.loc[mask_cbc,col]=pat.loc[mask_cbc,'hb']

    for col in crea_cols:
        mask = (pat.crea.isna())
        pat.loc[mask,col]=pat.loc[mask,'crea']

    for col in tsh_cols:
        mask = (pat.tsh.isna())
        pat.loc[mask,col]=pat.loc[mask,'tsh']
        
    pat.drop(['sick','diabetes','anemia','thyroid','total','crf'], axis=1, inplace=True)
    pat.set_index('id', inplace=True)
    order=['month','hb', 'rbc', 'wbc', 'leuk', 'lym', 'plt', 'crea', 'na', 'k', 'ca','gluc','tsh', 't4', 't3']
    return pat[order]

In [52]:
control=pd.DataFrame()
for i in range(100):
    temp=create_patient(i, [0.2,0.4,0.4],0.2)
    control=control.append(temp)
    
control.to_csv('control.csv')

In [53]:
case=pd.DataFrame()
for i in range(100):
    temp=create_patient(i, [0.1,0.3,0.6],0.3)
    case=case.append(temp)
    
case.to_csv('case.csv')